## Imports

In [21]:
import copy

import numpy as np
import gymnasium as gym
from IPython.display import HTML
from base64 import b64encode
import imageio

## Utils

In [22]:
def record_video(env, policy, out_directory, fps=1, random_action=False, max_steps=100):
    images = []  
    done = False
    truncated = False
    state, info = env.reset()
    img = env.render()
    images.append(img)
    total_reward = 0
    i = 0
    while not done and not truncated:
        i += 1
        if i > max_steps:
            break
        action = np.random.randint(4) if random_action else policy[state]
        state, reward, done, truncated, info = env.step(action)
        total_reward += reward
        img = env.render()
        images.append(img)
        if not random_action:
            print(f"action: {action}, state: {state}, reward: {reward}, done: {done}, truncated: {truncated}, info: {info}")
    imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)
    return total_reward

In [23]:
def show_video(video_path, video_width=500):
    video_file = open(video_path, "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

## Random Walk

In [24]:
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False, render_mode='rgb_array')
total_reward = record_video(env, None, 'frozenlake_random.mp4', fps=3, random_action=True)

show_video('frozenlake_random.mp4', video_width=500)


## Define FrozenLake MDP

In [25]:
class FrozenLakeMDP:
    def __init__(self, is_slippery):
        self.is_slippery = is_slippery
        self.terminal_states = np.zeros(16, dtype=int)
        self.terminal_states[[5, 7, 11, 12, 15]] = 1
        self.reward_fn = np.zeros(16, dtype=int)
        self.reward_fn[15] = 1

    def is_terminal(self, state):
        return self.terminal_states[state]
    
    def get_reward_function(self):
        return self.reward_fn

    def next_state_det(self, state, action):
        if action == 0:    # LEFT
            next_state = state - 1 if state % 4 != 0 else state
        elif action == 1:  # DOWN
            next_state = state + 4 if state // 4 != 3 else state
        elif action == 2:  # RIGHT
            next_state = state + 1 if state % 4 != 3 else state
        elif action == 3:  # UP
            next_state = state - 4 if state // 4 != 0 else state
        else:         # WRONG ACTION
            next_state = state
        return next_state
    
    def trans_prob(self, state, action):
        prob = np.zeros((16,), dtype=float)
        if not self.is_slippery:
            prob[self.next_state_det(state, action)] = 1.0
        else:
            prob[self.next_state_det(state, action)] += 1/3
            prob[self.next_state_det(state, (action+1)%4)] += 1/3
            prob[self.next_state_det(state, (action-1)%4)] += 1/3
        return prob

    def next_state_reward(self, state, action):
        next_state_probs = self.trans_prob(state, action)
        next_state = np.random.choice(16, p=next_state_probs)
        reward = self.reward_fn[next_state]
        return next_state, reward

In [26]:
dynamics = FrozenLakeMDP(is_slippery=False)

In [27]:
# reward function of the environent
dynamics.get_reward_function()

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [28]:
# evaluating if a given state is a terminal state (= hole or goal)
print(dynamics.is_terminal(0), dynamics.is_terminal(7), dynamics.is_terminal(15))

0 1 1


In [29]:
# if we take action `a` in state `s`,
# what is the probability of landing in each state?
dynamics.trans_prob(14, 2)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [30]:
# if we take action `a` in state `s`, what do we get?
# this is done through sampling the transition probability
next_state, reward = dynamics.next_state_reward(14, 2)
print(next_state, reward)

15 1


## Iterative Policy Evaluation

In [31]:
def policy_evaluation(dynamics, policy, gamma=0.9, num_iter=10):
    """
    evaluates policy based on Iterative Policy Evaluation.
 
    Args:
        dynamics (FrozenLakeMDP): dynamics of the environment.
        policy (numpy.array): policy we want to evaluate.
        gamma (float): discount factor.
        num_iter (int): number of iterations for the loop.
 
    Returns:
        numpy.array: state value function.
    """

    s_value_function = np.zeros(16, dtype=float)
    s_value_function[15] = dynamics.reward_fn[15]
    for _ in range(num_iter):
        ex_s_value_function = copy.copy(s_value_function)
        for s in range(16):
            if dynamics.is_terminal(s):
                continue
            action = policy[s]
            probs = dynamics.trans_prob(s, action)
            next_reward : int = 0
            for state in range(16):
                next_reward += probs[state] * (dynamics.reward_fn[s] + (gamma * ex_s_value_function[state]))
            if next_reward > s_value_function[s]:
                s_value_function[s] = next_reward
    return s_value_function

In [52]:
dynamics = FrozenLakeMDP(is_slippery=False)

# 1. go-right policy
policy = 2 * np.ones(16, dtype=int)

# 2. shortest-path policy
policy = np.array([1, 2, 1, 0, 1, -1, 1, -1, 2, 1, 1, -1, -1, 2, 2, -1])

s_value_function = policy_evaluation(dynamics, policy)

In [53]:
# TODO: print and analyze the state value function

for i in range(len(s_value_function)):
    print(i, " ---> ", s_value_function[i])

0  --->  0.5314410000000002
1  --->  0.5904900000000002
2  --->  0.6561000000000001
3  --->  0.5904900000000002
4  --->  0.5904900000000002
5  --->  0.0
6  --->  0.7290000000000001
7  --->  0.0
8  --->  0.6561000000000001
9  --->  0.7290000000000001
10  --->  0.81
11  --->  0.0
12  --->  0.0
13  --->  0.81
14  --->  0.9
15  --->  1.0


## Policy Iteration

In [54]:
def greedy_policy_improvement(dynamics, s_value_function, gamma=0.9):
    """
    obtains a policy in a greedy manner based on current state value function.
 
    Args:
        dynamics (FrozenLakeMDP): dynamics of the environment.
        s_value_function (numpy.array): state value function.
        gamma (float): discount factor.
 
    Returns:
        numpy.array: the greedy policy.
    """
    max_policy = np.zeros(shape=(16,), dtype=int)
    for state in range(16):
        if dynamics.is_terminal(state):
            continue
        state_reward = -100000000
        curr_policy = -1
        for action in range(4):
            probs : dict = dynamics.trans_prob(state, action)
            tmp = 0
            for i in range(16):
                tmp += probs[i] * (dynamics.reward_fn[state] + (gamma * s_value_function[i]))
            if tmp > state_reward:
                state_reward = tmp
                curr_policy = action
        max_policy[state] = curr_policy
            
    return max_policy

In [55]:
def policy_iteration(dynamics, gamma=0.9, outer_iter=1, inner_iter=100):
    """
    optimizes a policy based on Policy Iteration
 
    Args:
        dynamics (FrozenLakeMDP): dynamics of the environment.
        gamma (float): discount factor.
        outer_iter (int): number of iterations for the Policy Iteration loop.
        inner_iter (int): number of iterations for the Policy Evaluation loop.
 
    Returns:
        numpy.array: the optimized policy.
    """
    policy = np.array([1, 2, 1, 0, 1, -1, 1, -1, 2, 1, 1, -1, -1, 2, 2, -1], dtype=int)
    
    for _ in range(outer_iter):
        s_value_function = policy_evaluation(dynamics, policy, gamma, inner_iter)
        policy = greedy_policy_improvement(dynamics, s_value_function, gamma)

    return policy

In [60]:
# TODO: test and analyze the algorithm

dynamics = FrozenLakeMDP(is_slippery=True)
policy = policy_iteration(dynamics)
print(policy)

[0 3 0 3 0 0 0 0 3 1 0 0 0 2 1 0]


In [61]:
# TODO: test the policy on the environment

env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=True, render_mode='rgb_array')
total_reward = record_video(env, policy, 'frozenlake_random.mp4', fps=5, random_action=False)
print(f"total reward: {total_reward}")
show_video('frozenlake_random.mp4', video_width=500)

action: 0, state: 0, reward: 0.0, done: False, truncated: False, info: {'prob': 0.3333333333333333}
action: 0, state: 0, reward: 0.0, done: False, truncated: False, info: {'prob': 0.3333333333333333}
action: 0, state: 0, reward: 0.0, done: False, truncated: False, info: {'prob': 0.3333333333333333}
action: 0, state: 0, reward: 0.0, done: False, truncated: False, info: {'prob': 0.3333333333333333}
action: 0, state: 0, reward: 0.0, done: False, truncated: False, info: {'prob': 0.3333333333333333}
action: 0, state: 4, reward: 0.0, done: False, truncated: False, info: {'prob': 0.3333333333333333}
action: 0, state: 8, reward: 0.0, done: False, truncated: False, info: {'prob': 0.3333333333333333}
action: 3, state: 4, reward: 0.0, done: False, truncated: False, info: {'prob': 0.3333333333333333}
action: 0, state: 8, reward: 0.0, done: False, truncated: False, info: {'prob': 0.3333333333333333}
action: 3, state: 8, reward: 0.0, done: False, truncated: False, info: {'prob': 0.3333333333333333}


## Q-Learning

In [62]:
import numpy
class QAgent:  # The Q-Learning RL agent

    def __init__(self, num_states, num_actions, epsilon, alpha, gamma=0.9, eps_end=0.01, eps_decay=3e-6):

        self.num_states = num_states    # number of possible states
        self.num_actions = num_actions  # number of possible actions
        self.gamma = gamma              # discount factor
        self.epsilon = epsilon          # initial exploration probability
        self.alpha = alpha              # step size
        self.eps_decay = eps_decay      # linear decay rate of epsilon
        self.eps_end = eps_end          # minimum value for epsilon
        self.q_table = np.zeros((num_states, num_actions), dtype=float)

    def choose_action(self, state):
        """
        chooses an action in an epsilon-greedy manner.
    
        Args:
            state (int): current state of the agent.
    
        Returns:
            int: the chosen action
        """
        max_idx = numpy.argmax(self.q_table[state])
        p = [self.epsilon/self.num_actions for _ in range(self.num_actions)]
        p[max_idx] = 1 - (self.epsilon * (self.num_actions-1)) / self.num_actions
        action = numpy.random.choice(self.num_actions, p = p)

        return action

    def learn(self, state, action, reward, next_state):
        """
        updates the q-table based on a single interaction with the environment.
    
        Args:
            state (int): state of the agent.
            action (int): action chosen by the agent.
            reward (int): reward obtained by the agent.
            next_state (int): next state of the agent.
        """
        
        self.q_table[state, action] = self.alpha * (reward + max(self.q_table[next_state])) + (1-self.alpha) * self.q_table[state, action]
        
        # epsilon decay
        self.epsilon = self.epsilon - self.eps_decay if self.epsilon > self.eps_end else self.eps_end

In [63]:
def train(env, agent, n_episodes=100000):
    """
        trains an agent through interactions with the environemnt using Q-learning.
    
        Args:
            env (gym.Env): the gym environment.
            agent (QAgent): the Q-learning agent.
            n_episodes (int): number of training episodes.
    """
    env.reset()
    state = 0
    for _ in range(n_episodes):
        action = agent.choose_action(state=0)
        next_state, reward, terminated, truncated, _ = env.step(action)
        if terminated or truncated:
            env.reset()
        agent.learn(state, action, reward, next_state)
        state = next_state

In [64]:
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False)
agent = QAgent(num_states=16, num_actions=4, epsilon=1.0, alpha=1e-3)

In [66]:
train(env, agent)

1
2
0
2
2
2
0
3
2
2
2
2
3
2
0
2
2
1
2
0
2
0
2
0
2
2
2
3
2
0
1
1
1
0
3
0
2
0
2
2
2
3
2
1
2
2
2
2
0
2
0
3
2
2
2
2
1
2
2
1
3
0
0
2
0
2
2
0
1
2
0
0
1
2
0
2
2
3
3
3
3
2
3
1
2
3
2
2
2
2
1
2
2
2
3
1
0
2
2
2
2
0
2
1
1
1
2
0
1
0
0
3
1
1
3
1
2
2
2
3
2
2
1
2
1
3
2
1
2
2
1
2
2
1
2
2
2
3
3
0
2
3
0
2
2
2
3
0
2
0
3
2
1
1
2
1
2
0
2
2
2
0
2
2
2
2
1
2
2
2
1
2
0
3
3
1
2
2
2
3
2
1
2
2
2
2
1
1
3
0
2
2
2
2
3
1
3
2
2
2
2
2
1
3
2
1
2
0
3
1
2
2
1
2
2
2
3
2
3
2
2
2
1
0
2
2
1
2
2
2
0
3
0
2
0
1
2
1
3
1
1
0
3
2
2
2
2
0
3
2
0
2
1
2
3
0
2
1
1
2
3
0
1
3
2
1
3
2
2
2
1
0
3
3
2
0
2
3
2
2
2
2
2
0
0
2
2
2
2
2
2
1
3
2
0
2
2
2
1
3
2
1
2
2
2
2
0
1
2
2
2
3
2
1
1
2
2
0
1
1
2
1
2
2
0
2
1
1
0
3
3
0
2
3
2
2
1
3
2
2
0
2
2
2
3
2
2
2
1
2
1
2
0
0
1
0
2
3
0
3
2
2
2
2
3
2
2
1
2
1
0
2
2
2
1
3
1
2
1
0
0
0
0
0
3
2
1
2
1
2
1
2
3
0
2
2
3
0
0
2
2
1
1
0
2
2
3
1
2
1
2
1
3
2
2
2
0
2
2
2
3
0
2
0
3
1
3
3
2
1
2
2
1
0
2
1
3
2
2
2
2
2
2
2
3
0
2
3
3
0
0
1
2
3
2
0
2
2
3
3
3
3
0
1
2
2
2
2
0
2
2
2
2
1
2
2
2
1
3
1
2
0
2
1
1
2
2
0
2
1
2
3
2
2
0
0
2
3
1
0


In [67]:
policy = np.zeros(16, dtype=int)
for s in range(16):
    policy[s] = np.argmax(agent.q_table[s])



In [68]:
# TODO: test the policy on the environment

env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False, render_mode='rgb_array')
total_reward = record_video(env, policy, 'frozenlake_random.mp4', fps=5, random_action=False)
print(f"total reward: {total_reward}")
show_video('frozenlake_random.mp4', video_width=500)

action: 2, state: 1, reward: 0.0, done: False, truncated: False, info: {'prob': 1.0}
action: 2, state: 2, reward: 0.0, done: False, truncated: False, info: {'prob': 1.0}
action: 1, state: 6, reward: 0.0, done: False, truncated: False, info: {'prob': 1.0}
action: 1, state: 10, reward: 0.0, done: False, truncated: False, info: {'prob': 1.0}
action: 1, state: 14, reward: 0.0, done: False, truncated: False, info: {'prob': 1.0}
action: 2, state: 15, reward: 1.0, done: True, truncated: False, info: {'prob': 1.0}
total reward: 1.0
